In [ ]:
# @title 1. 한글 폰트 및 라이브러리 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf

# 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
pd.set_option('display.max_columns', None)

print("환경 설정 완료. 폰트 적용을 위해 런타임 재시작이 필요할 수 있습니다.")

In [ ]:
# @title 2. 데이터 전처리 및 회귀분석 수행

# 파일 경로 (업로드한 파일명과 일치해야 함)
file_kids = 'KOFOTI_통계보고서_24년-3월25년-2월-패션-소비-실태조사-1.xlsx - 2-8. 유아동복.csv'
file_casual = 'KOFOTI_통계보고서_24년-3월25년-2월-패션-소비-실태조사-1.xlsx - 2-3. 캐주얼복.csv'

# 데이터 추출 함수
def load_and_prep_data(file_path, category_label, is_kids_val):
    try:
        df = pd.read_csv(file_path, header=None)

        # 데이터가 시작되는 행(Row) 탐색 (보통 10행 이후 품목 데이터 시작)
        data = []
        start_row = 10

        for i in range(start_row, len(df)):
            item_name = str(df.iloc[i, 1]).strip()

            # 유효한 품목 행만 추출 (합계, 성별 구분 행 제외)
            if item_name == 'nan' or '전체' in item_name or '성별' in item_name or '연령' in item_name:
                continue

            try:
                # 컬럼 인덱스 확인 (파일 구조 기반)
                # Col 1: 품목명
                # Col 3: 평균 구매 개수 (Frequency)
                # Col 4: 평균 구입 금액 (Price)
                freq = float(str(df.iloc[i, 3]).replace(',', ''))
                price = float(str(df.iloc[i, 4]).replace(',', ''))

                if freq > 0 and price > 0:
                    data.append({
                        'Category': category_label,
                        'Item': item_name,
                        'Frequency': freq,
                        'Price': price,
                        'Is_Kids': is_kids_val # 더미 변수 (1: 아동복, 0: 성인복)
                    })
            except:
                continue
        return pd.DataFrame(data)
    except Exception as e:
        print(f"Error loading {category_label}: {e}")
        return pd.DataFrame()

# 1. 데이터 로드 및 병합
df_kids = load_and_prep_data(file_kids, 'Kids', 1)
df_adult = load_and_prep_data(file_casual, 'Adult', 0)

df_analysis = pd.concat([df_kids, df_adult], ignore_index=True)

# 2. 변수 변환 (로그 변환)
# 가격의 단위가 크고 분포가 편향되어 있으므로 로그 변환(Log-Transformation) 적용
df_analysis['ln_Price'] = np.log(df_analysis['Price'])

print(f"총 분석 데이터 수: {len(df_analysis)}개 품목")
print(df_analysis.head())

# 3. 회귀분석 모델링 (OLS Regression)
# 가설: Frequency = beta0 + beta1 * ln(Price) + beta2 * Is_Kids
print("\n>>> [회귀분석 결과] 모델: Frequency ~ ln(Price) + Is_Kids")
model = smf.ols(formula='Frequency ~ ln_Price + Is_Kids', data=df_analysis).fit()

print(model.summary())

# 4. 결과 시각화
plt.figure(figsize=(10, 6))

# 산점도 그리기
sns.scatterplot(x='Price', y='Frequency', hue='Category', style='Category',
                s=100, alpha=0.8, data=df_analysis)

# 회귀선 추가를 위한 예측값 생성 (시각화용)
# 성인복(Is_Kids=0)과 아동복(Is_Kids=1) 각각의 추세선
x_range = np.linspace(df_analysis['ln_Price'].min(), df_analysis['ln_Price'].max(), 100)
pred_adult = model.params['Intercept'] + model.params['ln_Price'] * x_range
pred_kids = model.params['Intercept'] + model.params['ln_Price'] * x_range + model.params['Is_Kids']

# x축을 다시 원래 가격 단위(exp)로 변환하여 그리기
plt.plot(np.exp(x_range), pred_adult, 'r--', label='성인복 추세선')
plt.plot(np.exp(x_range), pred_kids, 'b--', label='아동복 추세선')

plt.xscale('log') # X축 로그 스케일
plt.xlabel('평균 구입 가격 (원, Log Scale)')
plt.ylabel('연간 평균 구매 빈도 (회)')
plt.title('가격과 구매 빈도의 관계 (회귀분석 시각화)')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

# 5. 핵심 결과 출력
coef_price = model.params['ln_Price']
coef_kids = model.params['Is_Kids']
p_val_kids = model.pvalues['Is_Kids']

print("-" * 60)
print(f"[비즈니스 인사이트 요약]")
print(f"1. 가격 민감도 (ln_Price): {coef_price:.4f}")
print(f"   -> 가격이 1% 오를 때마다 구매 빈도는 감소하는 경향을 보임.")
print(f"2. 아동복 효과 (Is_Kids): {coef_kids:.4f} (P-value: {p_val_kids:.4f})")
if p_val_kids < 0.05:
    print(f"   -> 통계적으로 유의미함! 가격이 같다고 가정할 때, 아동